In [ ]:

!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset
from pprint import pprint



In [ ]:
import pickle
import re
from pprint import pformat

class Record:
    def __init__(self, claims, content, metadata):
        self.raw_claims = claims
        self.claims = list(self.raw_claims.split(','))
        self.raw_content = content
        self.metadata = metadata
        self.instances = []

    def pre_process_content(self):
        # Remove all content within angular brackets
        angular_brackets_pattern = re.compile(r'<.*?>')
        self.content = re.sub(angular_brackets_pattern, '', self.raw_content)

        # Remove newline characters
        newline_pattern = re.compile(r'\n')
        self.content = re.sub(newline_pattern, ' ', self.content)

    def split_content(self):
        # Split the content by the keywords "User:" and "Agent:"
        split_pattern = re.compile(r'(User:|Agent:)')
        segments = re.split(split_pattern, self.content)

        # Combine the keyword with its corresponding content
        combined_segments = []
        for i in range(1, len(segments), 2):
            combined_segments.append(segments[i] + ' ' + segments[i + 1].strip())

        self.split_content = combined_segments

    def __str__(self):
        output = f"Content: {self.content}\nInstances:\n"
        for instance in self.instances:
            output += f"{instance['claim']}:\n{pformat(instance['instances'])}\n"
        return output
records = pickle.load(open('/content/results_with_records_gpt4.pkl', 'rb'))


In [ ]:
SYSTEM_PROMPT = '''You are an advanced AI language model tasked with pinpointing specific instances in a conversation where a claim is discussed. Your objective is to thoroughly examine the dialogue between a user and an agent and identify all occurrences where the claim is mentioned. Each instance may appear in different segments of the conversation. Your response should present these instances in an array format, detailing their respective content that includes the given claim.

Chain of Thought:
1) Understand the Claim: Begin by carefully reading and comprehending the claim provided. Identify the core elements, key points, and main subject of the claim.
2) Identify Key Words and Synonyms: Look for important or key words in the claim and ensure these key words or their synonyms are present in the conversation.
3) Capture Relevant Phrases: After selecting the important key words, look for the phrases that encapsulate the entire claim and provide a clear understanding of the claim.
4) Verify Context and Relevance: Ensure that the identified phrases are relevant to the claim by analyzing their context within the conversation. Confirm that these phrases genuinely discuss the claim.
5) Validate Accuracy: Cross-check the selected phrases to ensure they accurately reflect the claim and are correctly interpreted. Verify that each instance aligns with the original claim's intent.
6) If no instances are found: Return an empty list in the `instances` key.

Instructions:
1) Review the Claim: Carefully examine the claim provided to understand its key points and main subject. Identify the essential elements of the claim.
2) Analyze the Conversation: Study the entire conversation between the user and the agent, which could be spread across multiple messages or parts. Look for the key words and synonyms related to the claim.
3) Identify Claim Mentions: Locate every instance where the claim is referenced within the conversation. Pay attention to context to ensure relevance.
4) Extract Relevant Phrases: Select the specific segments from the conversation that encapsulate the claim. Ensure these segments provide a clear understanding of the claim.
5) Output Format: Provide the output in JSON format as an array of phrases. The output should contain one key `instances`, which holds an array listing all unique instances where the claim is mentioned. If no instances are found, return an empty list in the `instances` key.

Guidelines:
1) Ensure Precise Matching: Make sure the selected phrases precisely match the key elements of the claim. No need to extract complete sentences; just select the sections from which the particular claim can be proved.
2) Use Logical Reasoning: Apply logical reasoning to accurately align each instance with the claim. Verify the context and relevance of each phrase.
3) Document Each Occurrence: Include each occurrence in the array with its encompassing content where the claim is discussed. Ensure no duplications in segments.
4) Maintain Clarity and Accuracy: Ensure the final output is clear, accurate, and logically aligned with the claim. Avoid ambiguity and ensure the claim is clearly reflected in the selected phrases.

Initiate your analysis:
'''

USER_PROMPT ='''Claim:
{claim}
Conversation:
{content}
Response (JSON Array):
'''


In [ ]:
from datasets import Dataset

dataset_dict = {'input':[], 'output':[], 'prompt':[]}

for record in records:
    for claim in record.instances:
        dataset_dict['input'].append(USER_PROMPT.format(claim=claim['claim'], content=record.content))
        dataset_dict['output'].append(claim['raw_instances'])
        dataset_dict['prompt'].append(SYSTEM_PROMPT)

dataset = Dataset.from_dict(dataset_dict)

TRAIN_LLM_INPUT_PROMPT ='''<s>[INST] <<SYS>> {system_prompt} <</SYS>> {user_msg} [/INST] {model_answer} </s>'''
INFER_LLM_INPUT_PROMPT ='''<s>[INST] <<SYS>> {system_prompt} <</SYS>> {user_msg} [/INST] {model_answer} </s>'''

dataset = dataset.map(lambda example: {'train_text': TRAIN_LLM_INPUT_PROMPT.format(system_prompt=example['prompt'], user_msg=example['input'], model_answer=example['output'])})
dataset = dataset.map(lambda example: {'infer_text': INFER_LLM_INPUT_PROMPT.format(system_prompt=example['prompt'], user_msg=example['input'], model_answer='')})


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

In [ ]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:

from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 16

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from transformers import TrainingArguments


In [ ]:
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 1
num_train_epochs = 4
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 20
warmup_ratio = 0.03
lr_scheduler_type = "linear"
output_dir = "/content/"


In [ ]:

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
from trl import SFTTrainer
max_seq_length = 4096


In [ ]:

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="train_text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 58.00 MiB. GPU 

In [ ]:

model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
text = dataset['text'][8]

device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

{
  "instances": [
    "I've noticed that we often get these 'duplicate' reasons where the titles are something like 'issue with [feature]', 'issue with [feature] on Feeld' and sometimes also 'issue with [feature] on Feeld app'.",
    "I assume this is because Enterpret doesn't know that Feeld is the only product that anyone might be referring to, so it tries to distinguish between customers specifically mentioning Feeld by name vs. just saying there's a problem with the app?",
    "that's correct. We are always working to improve the models so that they can better understand the context in which they are working, but it's not uncommon to have duplicate reasons like this."
  ]
}
```

### 2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2.2


In [ ]:
text

'{\n  "instances": [\n    "I\'ve noticed that we often get these \'duplicate\' reasons where the titles are something like \'issue with [feature]\', \'issue with [feature] on Feeld\' and sometimes also \'issue with [feature] on Feeld app\'.",\n    "I assume this is because Enterpret doesn\'t know that Feeld is the only product that anyone might be referring to, so it tries to distinguish between customers specifically mentioning Feeld by name vs. just saying there\'s a problem with the app?",\n    "that\'s correct. We are always working to improve the models so that they can better understand the context in which they are working, but it\'s not uncommon to have duplicate reasons like this."\n  ]\n}'